# Method

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics
from scipy.stats import norm
from myst_nb import glue
#plt.style.use('presentation')
from IPython.display import display, Markdown, Latex
import sympy as sp
from d2e2f.visualization.visualize import plot_map, plot_trips
import statsmodels.api as sm
import geopandas as gp
from d2e2f.pipelines.data_preprocessing.prepare import prepare
from operator import add
from functools import reduce

In [ ]:
%reload_kedro
loaders = catalog.load('vitaskar.raw_data')
renames = catalog.load('params:renames')
min_speed = catalog.load('params:vitaskar.min_speed')

loader = list(loaders.values())[6]
df_raw = loader()
#df_raw = df_raw.iloc[8900:9500].copy()
df_raw = df_raw.iloc[9500:20000].copy()

df_raw2 = df_raw.copy()
df_raw2.set_index("Timestamp", inplace=True)
df_raw2.index = pd.to_datetime(df_raw2.index)
df_raw2.sort_index(inplace=True)
df_raw2.index.name = "time"

In [ ]:
df_raw.head()

In [ ]:
df = prepare(df_raw, P_max=None, min_speed=min_speed, renames = renames)

In [ ]:
mask = ((df_raw2.index >= df.index[0]) & (df_raw2.index <= df.index[-1]))
df_raw2 = df_raw2.loc[mask].copy()


fig,ax=plt.subplots()
df_raw2['sog'] = df_raw2['Speed over ground (kts)']*1.852/3.6
df_raw2['limit'] = min_speed

df_raw2.plot(y='sog', style='.-', ax=ax)
df.plot(y='sog', style='.-', ax=ax)
df_raw2.plot(y='limit', style='-', ax=ax)


In [ ]:
mask = ((df_raw2['sog'] < min_speed))
mask[90:110]

In [ ]:
mask = ((np.roll(df_raw2['sog'],1) >= min_speed) & (df_raw2['sog'] < min_speed))
mask[90:110]

In [ ]:
mask = ((np.roll(df_raw2['sog'],1) < min_speed) & (df_raw2['sog'] >= min_speed))

In [ ]:
sog = df_raw2['sog']

mask_end = ((np.roll(sog,1) >= min_speed) & (sog < min_speed))
mask_start = ((sog < min_speed) & (np.roll(sog,-1) >= min_speed))

df_raw2.loc[mask_start,'state'] = 'start'
df_raw2.loc[mask_end,'state'] = 'end'


ends = df_raw2.loc[mask_end]
starts = df_raw2.loc[mask_start]

In [ ]:
fig,ax=plt.subplots()
df_raw2.plot(y='sog', ax=ax)
starts.plot(y='sog', style='sg', ax=ax)
ends.plot(y='sog', style='^r', ax=ax)

In [ ]:
#mask = df_raw2['state'].isin(['start','end'])
mask = df_raw2['state'].isin(['start'])

df_raw2['time'] = df_raw2.index
events = df_raw2.loc[mask]
time_limit = '300S'
events['time'].diff() < time_limit

In [ ]:
events

In [ ]:
mask = events['time'].diff() < time_limit

In [ ]:
events.loc[mask]

In [ ]:
iis = np.array([events.index.get_loc(row.name) for _,row in events.loc[mask].iterrows()])

In [ ]:
iis

In [ ]:
mask = events['time'].diff() < time_limit
#index = events.loc[mask].index
df_raw2.loc[events.loc[mask].index,'state'] = np.NaN
#iis = np.array([events.index.get_loc(row.name) for _,row in events.loc[mask].iterrows()])
#df_raw2.loc[events.iloc[iis-1].index,'state'] = np.NaN

#df_raw2.iloc[iis+1]['state'] = np.NaN

In [ ]:
mask = df_raw2['state']=='start'
df_raw2.loc[mask,'zero time'] = df_raw2.loc[mask,'time']
df_raw2['zero time'] = df_raw2['zero time'].fillna(method='ffill')
df_raw2['trip time'] = df_raw2['time'] - df_raw2['zero time']
mask = df_raw2['trip time'].notnull()
df_raw2=df_raw2.loc[mask]

In [ ]:
mask_end = ((np.roll(sog,1) >= min_speed) & (sog < min_speed) & (df_raw2['trip time'] > '600S'))
mask = df_raw2['state'] == 'end'
df_raw2.loc[mask,'state'] = np.NaN

df_raw2.loc[mask_end,'state'] = 'end'

In [ ]:
fig,ax=plt.subplots()
df_raw2.plot(y='sog', ax=ax)

for state, df__ in df_raw2.groupby(by='state'):
    if state is 'start':
        df__.plot(y='sog', style='sg', ax=ax)
    elif state is 'end':
        df__.plot(y='sog', style='^r', ax=ax)


In [ ]:
figs = []
figs.append(px.line(df_raw2, y='sog').data)

for state, df__ in df_raw2.groupby(by='state'):
    if state is 'start':
        figs.append(px.scatter(df__, y='sog',).update_traces(marker=dict(color='green')).data)
    elif state is 'end':
        figs.append(px.scatter(df__, y='sog').update_traces(marker=dict(color='red')).data)


fig = go.Figure(data=reduce(add, figs))
display(fig)

In [ ]:
df_raw2

In [ ]:
df = df_raw2
mask = df["state"] == "start"
starts = df.loc[mask]
mask = df["state"] == "end"
ends = df.loc[mask]


In [ ]:
len(starts)

In [ ]:
len(ends)

In [ ]:
starts.head()

In [ ]:
ends.head()

In [ ]:
if ends.index[0] < starts.index[0]:
    ends.drop(index=ends.iloc[0].name, inplace=True)
    
if ends.index[-1] < starts.index[-1]:
    starts.drop(index=starts.iloc[-1].name, inplace=True)


In [ ]:
mask = df["state"] == "start"
starts = df.loc[mask]

start_number = 0
end_number = start_number + 1 + len(starts)
trip_numbers = np.arange(start_number + 1, end_number, dtype=int)

df.loc[starts.index, "trip_no"] = trip_numbers
df["trip_no"] = df["trip_no"].fillna(method="ffill")
df["trip_no"] = df["trip_no"].fillna(start_number)
df["trip_no"] = df["trip_no"].astype(int)

In [ ]:
df

In [ ]:
df = catalog.load("vitaskar.preprocessed_data")

In [ ]:
df.iloc[0:1000]

In [ ]:
mask = df.index.duplicated(keep="first")
len(df.loc[~mask])

In [ ]:
df.loc[~mask]